In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import copy

# 기본 경로 설정
os.chdir('C:/Users/gksru/Desktop/study/Lpoint/제6회 L.POINT Big Data Competition')
# 폰트 설정
mpl.rc('font', family=' Malgun Gothic')
# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

warnings.filterwarnings('ignore')

### 1. 온라인 행동 정보<br>
* 데이터 설명서

|번호|변수명(영문)|변수명(국문)|상세 설명|PK|
|-|---|---|----|-|
|1|CLNT_ID|클라이언트 ID|고객을 고유하게 식별할 수 있도록 랜덤으로 부여된 ID|v|
|2|SESS_ID|세션 ID| Web/App 에 접속 후 세션이 시작될 때 부여된 순번 ID<br>※하나의 클라이언트 ID 에 여러 개의 세션 ID 가 발급될 수 있음|v|
|3|HIT_SEQ|조회일련번호|조회 순서를 알 수 있도록 부여된 일련번호|v|
|4|ACTION_TYPE|조회일련번호|총 8 가지의 행동 유형을 구분한 코드<br>[0.검색 / 1.제품 목록 / 2.제품 세부정보 보기<br> / 3.장바구니 제품 추가/4.장바구니 제품 삭제 / 5.결제 시도 <br>/ 6.구매 완료 / 7.구매 환불 / 8.결제 옵션 ]||
|5|BIZ_UNIT|업종단위|온라인 및 오프라인 이용처를 구분하는 단위코드||
|6|SESS_DT|세션일자|세션일자 (YYYYMMDD 형식으로 표시)||
|7|HIT_TM|조회시각|조회시각 (HH:MM 형식으로 표시)||
|8|HIT_PSS_TM|조회일련번호|세션이 시작된 이후 해당 조회까지 경과한 시간<br>(단위 : 밀리초)<br>※ 첫 번째 조회이면 0 으로 설정됩니다.||
|9|TRANS_ID|거래 ID|구매 내역을 고유하게 식별할 수 있도록 랜덤으로 부여된 ID||
|10|SRCH_KWD|검색 키워드|고객이 검색한 키워드||
|11|TOT_PAG_VIEW_CT|총페이지조회건수|세션 내의 총 페이지(화면)뷰 수||
|12|TOT_SESS_HR_V|총세션시간값|세션 내 총 시간(단위: 초)||
|13|TRFC_SRC|유입채널|고객이 유입된 채널<br>[DIRECT / PUSH / WEBSITE / PORTAL_1 / PORTAL_2 / PORTAL_3 / unknown]||
|14|DVC_CTG_NM|기기유형|기기 유형 [mobile_web / mobile_app / PC]||


In [3]:
data1 = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv')
data1.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
0,7809,1,8,5,A03,20190913,01:16,2571103,NaN,NaN,34.0,2663.0,DIRECT,NaN
1,7809,1,4,2,A03,20190913,01:14,2485909,NaN,NaN,34.0,2663.0,DIRECT,NaN
2,7809,1,11,5,A03,20190913,01:17,2646597,NaN,NaN,34.0,2663.0,DIRECT,NaN
3,7809,1,1,2,A03,20190913,00:46,788304,NaN,NaN,34.0,2663.0,DIRECT,NaN
4,7809,1,9,5,A03,20190913,01:17,2617609,NaN,NaN,34.0,2663.0,DIRECT,NaN


### 2. 거래 정보<br>
* 데이터 설명서

|번호|변수명(영문)|변수명(국문)|상세 설명|PK|
|-|---|---|----|-|
|1|CLNT_ID|클라이언트 ID|고객을 고유하게 식별할 수 있도록 랜덤으로 부여된 ID|v|
|2|TRANS_ID|거래 ID|구매 내역을 고유하게 식별할 수 있도록 랜덤으로 부여된 ID |v|
|3|TRANS_SEQ|거래일련번호|해당 거래 ID 의 구매 내역 내 구매 순서를 알 수 있도록<br> 부여된 일련번호|v|
|4|BIZ_UNIT|업종단위|온라인 및 오프라인 이용처를 구분하는 단위코드||
|5|PD_C|상품소분류코드|상품의 소분류코드||
|6|DE_DT|구매일자|상품을 구매한 일자 (YYYYMMDD 형식)||
|7|DE_TM|구매시각|상품을 구매한 시각 (HHMM 형식)||
|8|BUY_AM|구매금액|구매한 상품의 금액||
|9|BUY_CT|구매수량|구매한 상품의 수량||


In [4]:
data2 = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv')
data2.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1
3,18362,104010,1,A03,unknown,20190920,09:57,9900,1
4,39423,105124,13,A03,0565,20190920,17:26,2990,1


### 3. 고객 Demographic 정보<br>
* 데이터 설명서

|번호|변수명(영문)|변수명(국문)|상세 설명|PK|
|-|---|---|----|-|
|1|CLNT_ID|클라이언트 ID|고객을 고유하게 식별할 수 있도록 랜덤으로 부여된 ID|v|
|2|CLNT_GENDER|성별|성별정보<br> [남자 : M / 여자 : F / 정보없음 : unknown]||
|3|CLNT_AGE|연령대|연령대 정보<br>[10 대이하 / 20 대 / 30 대 / 40 대 / 50 대 / 60 대이상<br> / 정보없음 : unknown]||

In [6]:
data3 = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv')
data3.head()

,clnt_id,clnt_gender,clnt_age
0,1,unknown,unknown
1,2,F,30
2,3,unknown,unknown
3,4,unknown,unknown
4,5,unknown,unknown


### 4. 상품 분류 정보<br>
* 데이터 설명서

|번호|변수명(영문)|변수명(국문)|상세 설명|PK|
|-|---|---|----|-|
|1|PD_C|상품 소분류코드|상품의 소분류코드(최소단위)|v|
|2|CLAC1_NM|상품 대분류명|상품 대분류명||
|3|CLAC2_NM|상품 중분류명|상품 중분류명||
|4|CLAC3_NM|상품 소분류명|상품 소분류명||

In [7]:
data4 = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv')
data4.head()

,pd_c,clac_nm1,clac_nm2,clac_nm3
0,1,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1,2,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights
2,3,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint
3,4,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Filters
4,5,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Wiper Blades
